In [19]:
import pandas as pd
import datetime

In [23]:
td = datetime.timedelta(days=100*365)

In [51]:
t+td+td+ datetime.timedelta(days=95*365)

datetime.timedelta(days=36500)

In [4]:
t.to_numpy()

numpy.datetime64('1970-01-01T00:00:20.000000000')

In [5]:
int(t.to_numpy())/1000000000

20.0

In [12]:
pd.Timedelta(seconds=100000)

Timedelta('1 days 03:46:40')

In [54]:
pd.to_datetime('1980-01-01')

AttributeError: attribute 'year' of 'datetime.date' objects is not writable